In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) nn
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.1})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Determine the comment sentiment polarity in the given review delimited by triple quotes. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Only answer with a format of [\"sentiment\"] without any explanation. If the polarity can't be determined just answer [\"neutral\"].
               ```{text}```
           """

# template = """
# Determine the text sentiment according to the options : Positive, Neutral, and Negative, answer without explaination and you must choose one of the answer.
# ```{text}```
# """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)



In [ ]:
import pandas as pd
df = pd.read_excel('fix_fixed.xlsx')
df

,Data,Label,translated,Unnamed: 3,review_text_cleaned
0,bangka blm bukan kantor pos ambil nya,1,bangka blm bukan kantor pos ambil nya,NaN,bangka belum bukan kantor pos ambil nya
1,halo min apa kpr sudah berjalan bisa minta di...,2,lingkaran cahaya min apa kpr sudah berjalan b...,NaN,lingkaran cahaya min apa kpr sudah berjalan bi...
2,perancis,1,perancis,NaN,perancis
3,kapan giveaway nya nih megamin [smirking face],0,kapan memberi secara gratis nya nih megamin w...,NaN,kapan memberi secara gratis nya ini megamin wa...
4,geh yang balam tau gak soto tangkar yang dulu ...,1,geh yang balam tau gak soto tangkar yang dulu...,NaN,geh yang balam tahu tidak soto tangkar yang du...
...,...,...,...,...,...
31994,kak brarti perlu download welma lagi kan utk b...,0,kak brarti perlu unduh welma lagi kan utk beli...,NaN,kak berarti perlu unduh welma lagi kan untuk b...
31995,semoga kita terhindar dari modus penipuan bri ...,0,semoga kita terhindar dari modus penipuan bri ...,NaN,semoga kita terhindar dari modus penipuan bri ...
31996,hah dimana tuh tahun kemaren ngajuin mintanya...,2,ha dimana tuh tahun kemaren ngajuin mintanya 2jt,NaN,tertawa di mana itu tahun kemarin ngajuin mint...
31997,masang nomor mudah mudahan aja bisa tembus tuh...,0,masang nomor mudah mudahan aja bisa tembus tuh...,NaN,memasang nomor mudah mudahan saja bisa tembus ...


In [ ]:
df = df[['Data','Label']]
df['Data'] = df['Data'].astype(str)

<ipython-input-9-f2de878205cd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data'] = df['Data'].astype(str)


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
test_df = test_df.head(2000)

## Prompt Engineering LLaMa 2

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """Classify the text sentiment into neutral, negative, or positive. Reply with only one word: Positive, Negative, or Neutral.

Examples:
Text: masa koreksi pada pertumbuhan saat ini ternyata masih ada bank dengan laba bersih yang meningkat salah satunya bank btn bank btn membukukan laba bersih sebesar triliun pada tahun 2020
Sentiment: Positive.

Text: wajar menghujat karena ada alasannya sudah janji janji palsunya omongannya seperti benar sendiri kalau mau hujat bicara benar janji tepati
Sentiment: Negative.

Text: {text}
Sentiment:"""

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
coba = 'kali muncul logo brimo'
print(llm_chain.run(coba))

 Neutral.


In [ ]:
def classify(text):
  template = """Classify the text into neutral, negative, or positive. Reply with only one word: Positive, Negative, or Neutral.

  Examples:
  Text: Big variety of snacks (sweet and savoury) and very good espresso Machiatto with reasonable prices, you can't get wrong if you choose the place for a quick meal or coffee.
  Sentiment: Positive.

  Text: I got food poisoning
  Sentiment: Negative.

  Text: {text}
  Sentiment:"""

  prompt = PromptTemplate(template=template, input_variables=["text"])

  llm_chain = LLMChain(prompt=prompt, llm=llm)
  raw_llm_answer = llm_chain.run(text)
  llm_answer = raw_llm_answer.lower()
  # print(llm_answer,'text : ', text)
  if "neutral" in llm_answer:
      return 1
  elif "positive" in llm_answer:
      return 0
  elif "negative" in llm_answer:
      return 2
  else:
      return 1

In [ ]:
from tqdm import tqdm
jawab  = []
for text in tqdm(test_df['Data']):
  ans = classify(text)
  jawab.append(ans)

  0%|          | 9/2000 [00:11<41:25,  1.25s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2000/2000 [51:10<00:00,  1.54s/it]


In [ ]:
test_df['predicted'] = jawab

In [ ]:
correct_predictions = (test_df['Label'] == test_df['predicted']).sum()

# Calculate accuracy
total_predictions = len(test_df)
accuracy = correct_predictions / total_predictions

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 37.25%
